In [15]:
 !pip install rioxarray "xarray[complete]" geopandas

In [16]:
import rioxarray as rx
import pandas as pd
import boto3
import xarray as xr
import glob
import shutil
import os
import zarr
import s3fs
import geopandas as gpd
from dask.diagnostics import ProgressBar

In [17]:
pathS3 = "s3://climate-action-datalake/zone=raw/source=agera5"
localPath = 'data/*.nc'

In [18]:
localPath

'data/*.nc'

In [19]:
allFiles = xr.open_mfdataset(localPath,parallel=True)

In [20]:
allFiles

<xarray.Dataset>
Dimensions:              (time: 15706, lon: 3600, lat: 1801)
Coordinates:
  * time                 (time) datetime64[ns] 1980-01-01 ... 2022-12-31
  * lon                  (lon) float64 -180.0 -179.9 -179.8 ... 179.8 179.9
  * lat                  (lat) float64 90.0 89.9 89.8 89.7 ... -89.8 -89.9 -90.0
Data variables:
    Wind_Speed_10m_Mean  (time, lat, lon) float32 dask.array<chunksize=(1, 1801, 3600), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 1.9.2 (http://mpimet.mpg.de/...
    history:      Fri Mar 12 13:30:21 2021: cdo splitday /archive/ESG/wit015/...
    Conventions:  CF-1.7
    CDO:          Climate Data Operators version 1.9.2 (http://mpimet.mpg.de/...

In [21]:
compressor = zarr.Blosc(cname='zstd')
encoding = {vname: {'compressor': compressor,'chunks': (1,1801,3600)} for vname in allFiles.data_vars}

In [22]:
compressor

Blosc(cname='zstd', clevel=5, shuffle=SHUFFLE, blocksize=0)

In [23]:
s3 = s3fs.S3FileSystem()
store = s3fs.S3Map(root="s3://climate-action-datalake/zone=raw/source=agera5/variable=relativehumidity.zarr", s3=s3, check=False)

In [24]:
s3
store

In [25]:
with ProgressBar():
    result = allFiles.to_zarr(store=store, encoding=encoding, consolidated=True)

[########################################] | 100% Completed | 43m 19s
